<a href="https://colab.research.google.com/github/rikurzt/Finacial_analyze_course_HW/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_HW10_7113029027%E6%9D%8E%E5%86%A0%E5%BB%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料前處理

In [6]:
from torch.utils.data import DataLoader,dataset

In [7]:
data_dir = "/content/Sentences_50Agree.txt"

In [8]:
def read_file(path):
  with open(path, "r", encoding="latin-1") as file:
    docus = file.readlines()
    newDocus = []
    for data in docus:
      newDocus.append(data)

    return newDocus

In [9]:
import os
import torch

In [10]:
All_Content = read_file(data_dir)

In [11]:
#Train test split
from sklearn.model_selection import train_test_split
trainContent,testContent = train_test_split(All_Content,test_size=0.2,random_state=42)

In [12]:
label_map = {"neutral": 0, "positive": 1, "negative": 2}

In [13]:
class Label_Dataset(dataset.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, ind):
        onetext = self.data[ind]
        content, label = onetext.split("@")
        label = label_map[label.strip()]  # 使用映射字典轉換為數值
        label = torch.LongTensor([label])  # 模型需要 longTensor 才能運作
        return content, label

In [14]:
traindataset = Label_Dataset(trainContent)
testdataset = Label_Dataset(testContent)
testdataset.data[2]

'In 2009 , Lee & Man had a combined annual production capacity of close to 4.5 million tonnes of paper and 300,000 tonnes of pulp .@neutral\n'

In [15]:
traindataloader = DataLoader(traindataset, batch_size=20, shuffle=True)
testdataloader = DataLoader(testdataset, batch_size=200, shuffle=False)

In [16]:
print(len(traindataloader))

194


In [17]:
class_list = ["negative","neutral","positive"]

# 實作模型

In [18]:
from transformers import BertTokenizer,AutoConfig,AutoModelForSequenceClassification

In [19]:
pretrained_weights = "bert-base-uncased"

In [20]:
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
config = AutoConfig.from_pretrained(pretrained_weights,num_labels=len(class_list))#直接使用pretrain的config
model = AutoModelForSequenceClassification.from_pretrained(pretrained_weights,config=config).cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

AdamW和WarmUp

In [22]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

In [23]:
epoch = 2

設定optimizer

In [24]:

param_opt = list(model.named_parameters())
no_decay = ["bias","LayerNorm.bias","LayerNorm.weight"]
optimizer_group_paramaters=[
    {"params":[p for n,p in param_opt if not any(nd in n for nd in no_decay)],"weight_decay":0.01},
    {"params":[p for n,p in param_opt if any(nd in n for nd in no_decay)],"weight_decay":0.0},
]
optimizer = AdamW(optimizer_group_paramaters,lr = 5e-5,eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=100,num_training_steps=len(traindataloader)*epoch)

訓練步驟

In [25]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score

In [26]:
#思路，因為使用pre-train，所以不需要跑太多訓練效果就很好，所以再進行多少次訓練的batch後就進行一次evaluate
def evaluate(model,dataloader):
  model.eval()#模型切換到evaluate mode
  loss_total = 0

  predict_all = np.array([],dtype=int)#暫存每個batch的資料
  labels_all = np.array([],dtype=int)

  with torch.no_grad():#eval狀態下不計算梯度
    for sku_name,labels in testdataloader:
      ids = tokenizer.batch_encode_plus(sku_name,padding = 'max_length',return_tensors='pt') #把句子轉換成token，因為句子有長有短，所以要做padding，並指定返回的東西是pytorch的tensor
      labels = labels.squeeze().cuda()#batch中的label是一個直的2D tensor，需要變成1D
      input_ids = ids['input_ids'].cuda()
      attention_mask = ids['attention_mask'].cuda()#tokenizer做好的

      output = model(input_ids.cuda(),labels=labels.cuda(),attention_mask = attention_mask.cuda())
      loss,logits = output[:2]
      loss_total += loss.item()

      labels = labels.data.cpu().numpy()#labels在GPU內，需先放回CPU
      predict = torch.argmax(logits,axis=1).data.cpu().numpy()
      labels_all = np.append(labels_all,labels)
      predict_all = np.append(predict_all,predict)
    acc = accuracy_score(labels_all,predict_all)
    return acc , loss_total / len(testdataloader)


In [27]:
total_batch= 0
for epoch in range(epoch):
  for i,(sku_name,label) in enumerate(traindataloader):

      ids = tokenizer.batch_encode_plus(sku_name,padding = 'max_length',return_tensors='pt')
      labels = label.squeeze().cuda()
      input_ids = ids['input_ids'].cuda()
      attention_mask = ids['attention_mask'].cuda()

      output = model(input_ids,labels=labels,attention_mask = attention_mask)
      loss,logits = output[:2]
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(),0.1)#將算出來的梯度做normalize
      optimizer.step()
      scheduler.step()
      model.zero_grad()
      if(total_batch%10 == 0):
        truelabel = labels.data.cpu()#會使用sklearn，所以在CPU算即可
        predict = torch.argmax(logits,axis=1).data.cpu().numpy()#會使用sklearn，所以在CPU算即可
        train_acc = accuracy_score(truelabel,predict)
        dev_acc,dev_loss = evaluate(model,testdataloader)#執行測試
        msg ="itr:{0:>6},train loss:{1:5.2},train acc{2:>6.2},test loss:{1:5.2},test acc{2:>6.2}"
        print(msg.format(total_batch,loss.item(),train_acc,dev_loss,dev_acc))
        model.train()#要記得改回去訓練模式，不然不會被更新
      total_batch += 1


itr:     0,train loss:  1.1,train acc   0.2,test loss:  1.1,test acc   0.2
itr:    10,train loss:  1.1,train acc   0.2,test loss:  1.1,test acc   0.2
itr:    20,train loss: 0.92,train acc   0.6,test loss: 0.92,test acc   0.6
itr:    30,train loss:  0.9,train acc   0.7,test loss:  0.9,test acc   0.7
itr:    40,train loss: 0.73,train acc   0.7,test loss: 0.73,test acc   0.7
itr:    50,train loss:  1.0,train acc   0.5,test loss:  1.0,test acc   0.5
itr:    60,train loss: 0.71,train acc   0.6,test loss: 0.71,test acc   0.6
itr:    70,train loss: 0.57,train acc  0.65,test loss: 0.57,test acc  0.65
itr:    80,train loss: 0.92,train acc  0.55,test loss: 0.92,test acc  0.55
itr:    90,train loss: 0.46,train acc  0.85,test loss: 0.46,test acc  0.85
itr:   100,train loss: 0.35,train acc  0.95,test loss: 0.35,test acc  0.95
itr:   110,train loss:  0.6,train acc  0.75,test loss:  0.6,test acc  0.75
itr:   120,train loss: 0.31,train acc   0.9,test loss: 0.31,test acc   0.9
itr:   130,train loss: 0.

In [28]:
import random

In [35]:
def test_model_with_random_data(model, testdataset, tokenizer, class_list):

  random_index = random.randint(0, len(testdataset) - 1)
  content, label = testdataset[random_index]

  ids = tokenizer.encode_plus(
      content,
      padding="max_length",
      return_tensors="pt"
  )

  input_ids = ids['input_ids'].cuda()
  attention_mask = ids['attention_mask'].cuda()

  with torch.no_grad():
    model.eval()
    output = model(input_ids, attention_mask=attention_mask)
    logits = output.logits

    predicted_label = torch.argmax(logits, dim=1).item()

  print("真實標籤:", class_list[label.item()])
  print("預測標籤:", class_list[predicted_label])
  print("句子:", content)


for i in range(3):
  test_model_with_random_data(model, testdataset, tokenizer, class_list)

真實標籤: negative
預測標籤: negative
句子: These module products will be available for trials during 3Q-07 and for volume deliveries during 4Q-07 .
真實標籤: negative
預測標籤: negative
句子: The Remote Radio head module will be available at 4W power for 2.5 GHz and 3.5 GHz TDD frequency bands .
真實標籤: positive
預測標籤: positive
句子: The Finnish company previously said its operating result will be lower than the break-even posted a year earlier .
